<a href="https://colab.research.google.com/github/Gstv2711/projetocomp/blob/main/Vers%C3%A3o_24_05_2021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from ast import literal_eval
from sklearn.feature_extraction.text import CountVectorizer


# In[2]:


filmes = pd.read_csv("movies_metadata.csv")
creditos = pd.read_csv("credits.csv")

#Usando as palavras chaves no dataset keywords

palavraschaves = pd.read_csv("keywords.csv")


# In[3]:


filmes.head()


# In[4]:


filmes = filmes[["id", "title", "genres"]]


# In[5]:


#Função de limpar os ID's

def limpa_id(x):
    
    
    try: 
        
        return int(x)
    
    except:
        
        
        return np.nan
    
    


# In[6]:


filmes.head()


# In[7]:


filmes['id'] = filmes['id'].apply(limpa_id)
filmes = filmes[filmes['id'].notnull()]


# In[8]:


filmes['id'] = filmes['id'].astype('int')
palavraschaves['id'] = palavraschaves['id']
creditos['id'] = creditos['id'].astype('int')


# In[9]:


#Juntando através do id

filmes1 = filmes.merge(creditos, on = "id")
filmes2 = filmes1.merge(palavraschaves, on = "id")


# In[10]:


filmes.head()


# In[11]:


filmes1.head()


# In[12]:


filmes2.head()


# In[13]:


creditos.head()


# In[14]:


# aplicando literal_eval para fazer string -> objetos.

filmes2['genres'] = filmes2['genres'].apply(literal_eval)


# In[15]:


filmes2.head()


# In[17]:


filmes2.info()


# In[18]:


filmes2['cast'] = filmes2['cast'].apply(literal_eval)


# In[19]:


filmes2['crew'] = filmes2['crew'].apply(literal_eval)
filmes2['keywords'] = filmes2['keywords'].apply(literal_eval)


# In[20]:


#pegando o nome dos generos associados
filmes2['crew'] = filmes2['crew'].apply(lambda x: [i['name'].lower() for i in x if i['job'] == 'Director'])


# In[25]:


#juntando com as palavras chaves
df = filmes2


# In[26]:


df.head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 46628 entries, 0 to 46627
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        46628 non-null  int64 
 1   title     46624 non-null  object
 2   genres    46628 non-null  object
 3   cast      46628 non-null  object
 4   crew      46628 non-null  object
 5   keywords  46628 non-null  object
dtypes: int64(1), object(5)
memory usage: 2.5+ MB


,id,title,genres,cast,crew,keywords
0,862,Toy Story,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...","[{'cast_id': 14, 'character': 'Woody (voice)',...",[john lasseter],"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,Jumanji,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...","[{'cast_id': 1, 'character': 'Alan Parrish', '...",[joe johnston],"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,Grumpier Old Men,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...","[{'cast_id': 2, 'character': 'Max Goldman', 'c...",[howard deutch],"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,31357,Waiting to Exhale,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...","[{'cast_id': 1, 'character': 'Savannah 'Vannah...",[forest whitaker],"[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,11862,Father of the Bride Part II,"[{'id': 35, 'name': 'Comedy'}]","[{'cast_id': 1, 'character': 'George Banks', '...",[charles shyer],"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


In [ ]:
df["genres"] = df["genres"].apply(lambda x: [i["name"].lower() for i in x])

In [ ]:
df.head()

,id,title,genres,cast,crew,keywords,metadata
0,862,Toy Story,"[animation, comedy, family]","[tomhanks, timallen, donrickles]",[johnlasseter],"[jealousy, toy, boy]",animation comedy family tomhanks timallen donr...
1,8844,Jumanji,"[adventure, fantasy, family]","[robinwilliams, jonathanhyde, kirstendunst]",[joejohnston],"[boardgame, disappearance, basedonchildren'sbook]",adventure fantasy family robinwilliams jonatha...
2,15602,Grumpier Old Men,"[romance, comedy]","[waltermatthau, jacklemmon, ann-margret]",[howarddeutch],"[fishing, bestfriend, duringcreditsstinger]",romance comedy waltermatthau jacklemmon ann-ma...
3,31357,Waiting to Exhale,"[comedy, drama, romance]","[whitneyhouston, angelabassett, lorettadevine]",[forestwhitaker],"[basedonnovel, interracialrelationship, single...",comedy drama romance whitneyhouston angelabass...
4,11862,Father of the Bride Part II,[comedy],"[stevemartin, dianekeaton, martinshort]",[charlesshyer],"[baby, midlifecrisis, confidence]",comedy stevemartin dianekeaton martinshort cha...


In [ ]:
df["cast"] = df["cast"].apply(lambda x: [i["name"].lower() for i in x])


In [ ]:
df["keywords"] = df["keywords"].apply(lambda x: [i["name"].lower() for i in x])


TypeError: ignored

In [ ]:
#pegando até 3 caracteristicas de cada filme
# se pegarmos muitas caracteristicas pode aumentar demais a complexidade do algortimo

df["genres"] = df["genres"].apply(lambda x: x[:3] if len(x)>3 else x)
df["cast"] = df["cast"].apply(lambda x: x[:3] if len(x)>3 else x)
df["keywords"] = df["keywords"].apply(lambda x: x[:3] if len(x)>3 else x)

In [ ]:
df.head()

,id,title,genres,cast,crew,keywords
0,862,Toy Story,"[animation, comedy, family]","[tom hanks, tim allen, don rickles]",[john lasseter],"[jealousy, toy, boy]"
1,8844,Jumanji,"[adventure, fantasy, family]","[robin williams, jonathan hyde, kirsten dunst]",[joe johnston],"[board game, disappearance, based on children'..."
2,15602,Grumpier Old Men,"[romance, comedy]","[walter matthau, jack lemmon, ann-margret]",[howard deutch],"[fishing, best friend, duringcreditsstinger]"
3,31357,Waiting to Exhale,"[comedy, drama, romance]","[whitney houston, angela bassett, loretta devine]",[forest whitaker],"[based on novel, interracial relationship, sin..."
4,11862,Father of the Bride Part II,[comedy],"[steve martin, diane keaton, martin short]",[charles shyer],"[baby, midlife crisis, confidence]"


In [ ]:
#removendo os espaços

df["cast"] = df["cast"].apply(lambda x: [i.replace(" ","") for i in x])
df["crew"] = df["crew"].apply(lambda x: [i.replace(" ","") for i in x])
df["keywords"] = df["keywords"].apply(lambda x: [i.replace(" ","") for i in x])
df["genres"] = df["genres"].apply(lambda x: [i.replace(" ","") for i in x])

In [ ]:
df.head()

,id,title,genres,cast,crew,keywords
0,862,Toy Story,"[animation, comedy, family]","[tomhanks, timallen, donrickles]",[johnlasseter],"[jealousy, toy, boy]"
1,8844,Jumanji,"[adventure, fantasy, family]","[robinwilliams, jonathanhyde, kirstendunst]",[joejohnston],"[boardgame, disappearance, basedonchildren'sbook]"
2,15602,Grumpier Old Men,"[romance, comedy]","[waltermatthau, jacklemmon, ann-margret]",[howarddeutch],"[fishing, bestfriend, duringcreditsstinger]"
3,31357,Waiting to Exhale,"[comedy, drama, romance]","[whitneyhouston, angelabassett, lorettadevine]",[forestwhitaker],"[basedonnovel, interracialrelationship, single..."
4,11862,Father of the Bride Part II,[comedy],"[stevemartin, dianekeaton, martinshort]",[charlesshyer],"[baby, midlifecrisis, confidence]"


In [ ]:
#Transformando tudo em uma coluna só 

df["metadata"] = df.apply(lambda x : " ".join(x["genres"]) + " "  + " ".join(x["cast"]) + " " + " ".join(x["crew"]) + " " + " ".join(x["keywords"]), axis = 1)

In [ ]:
df.head()

,id,title,genres,cast,crew,keywords,metadata
0,862,Toy Story,"[animation, comedy, family]","[tomhanks, timallen, donrickles]",[johnlasseter],"[jealousy, toy, boy]",animation comedy family tomhanks timallen donr...
1,8844,Jumanji,"[adventure, fantasy, family]","[robinwilliams, jonathanhyde, kirstendunst]",[joejohnston],"[boardgame, disappearance, basedonchildren'sbook]",adventure fantasy family robinwilliams jonatha...
2,15602,Grumpier Old Men,"[romance, comedy]","[waltermatthau, jacklemmon, ann-margret]",[howarddeutch],"[fishing, bestfriend, duringcreditsstinger]",romance comedy waltermatthau jacklemmon ann-ma...
3,31357,Waiting to Exhale,"[comedy, drama, romance]","[whitneyhouston, angelabassett, lorettadevine]",[forestwhitaker],"[basedonnovel, interracialrelationship, single...",comedy drama romance whitneyhouston angelabass...
4,11862,Father of the Bride Part II,[comedy],"[stevemartin, dianekeaton, martinshort]",[charlesshyer],"[baby, midlifecrisis, confidence]",comedy stevemartin dianekeaton martinshort cha...


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 46628 entries, 0 to 46627
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        46628 non-null  int64 
 1   title     46624 non-null  object
 2   genres    46628 non-null  object
 3   cast      46628 non-null  object
 4   crew      46628 non-null  object
 5   keywords  46628 non-null  object
 6   metadata  46628 non-null  object
dtypes: int64(1), object(6)
memory usage: 2.8+ MB


In [ ]:
df_metadata = df.iloc[:20000, 6]

In [ ]:
df_metadata

0        animation comedy family tomhanks timallen donr...
1        adventure fantasy family robinwilliams jonatha...
2        romance comedy waltermatthau jacklemmon ann-ma...
3        comedy drama romance whitneyhouston angelabass...
4        comedy stevemartin dianekeaton martinshort cha...
                               ...                        
19995    comedy romance horror aliciasilverstone kryste...
19996    crime drama petercushing andrémorell richardve...
19997                                         cosmojarvis 
19998    horror sarahbolger lilycole sarahgadon maryhar...
19999    romance crime drama veronicakedar sivanlevy ro...
Name: metadata, Length: 20000, dtype: object

In [ ]:
#Usando cosseno de similaridade para achar a semelhança entre dois filmes
#somente necessário para a função sistema()

cv = CountVectorizer(stop_words = 'english')

contador_matrix = cv.fit_transform(df_metadata)

cosseno_sim_matrix = cosine_similarity(contador_matrix)

In [ ]:
cosseno_sim_matrix

array([[1.        , 0.09534626, 0.1       , ..., 0.        , 0.        ,
        0.        ],
       [0.09534626, 1.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.1       , 0.        , 1.        , ..., 0.        , 0.        ,
        0.09534626],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.09534626, ..., 0.        , 0.        ,
        1.        ]])

In [ ]:
len(cosseno_sim_matrix)

20000

In [ ]:
#Mapeando a função 

mapear = pd.Series(df_metadata.index, index = df.iloc[:20000, 1])
mapear

title
Toy Story                          0
Jumanji                            1
Grumpier Old Men                   2
Waiting to Exhale                  3
Father of the Bride Part II        4
                               ...  
Vamps                          19995
Cash on Demand                 19996
The Naughty Room               19997
The Moth Diaries               19998
Joe + Belle                    19999
Length: 20000, dtype: int64

In [ ]:
df_metadata

0        animation comedy family tomhanks timallen donr...
1        adventure fantasy family robinwilliams jonatha...
2        romance comedy waltermatthau jacklemmon ann-ma...
3        comedy drama romance whitneyhouston angelabass...
4        comedy stevemartin dianekeaton martinshort cha...
                               ...                        
19995    comedy romance horror aliciasilverstone kryste...
19996    crime drama petercushing andrémorell richardve...
19997                                         cosmojarvis 
19998    horror sarahbolger lilycole sarahgadon maryhar...
19999    romance crime drama veronicakedar sivanlevy ro...
Name: metadata, Length: 20000, dtype: object

In [ ]:
# Funcao de recomendacao de acordo com nossos metadados

def sistema(filmes_input):
    
  filme_indexado = mapear[filmes_input]
    
  #obtendo os valores similares
    
  score = list(enumerate (cosseno_sim_matrix[filme_indexado]))
    
  score = sorted(score, key = lambda x: x[1], reverse = True)
    
  # Amostra de 20 filmes
    
    
  score = score[1:20]
    
  indices = [i[0] for i in score]
  
  #return (cosseno_sim_matrix[filme_indexado])
  return (mapear.iloc[indices])


In [ ]:
def sistema2(filmes_input):
  
  #"Somando os filmes"
  indice = 20000
  df_metadata[indice] = ' '
  for filme in filmes_input:
    df_metadata[indice] += ' ' + df_metadata[mapear[filme]]

  #Usando a similaridade de cosseno
  cv = CountVectorizer(stop_words = 'english')
  contador_matrix = cv.fit_transform(df_metadata)
  cosseno_sim_matrix = cosine_similarity(contador_matrix)

  #Resetando a soma de filmes
  df_metadata[indice] = ' '

  #obtendo os valores similares
  
  score = list(enumerate(cosseno_sim_matrix[indice]))
  score = sorted(score, key = lambda x: x[1], reverse = True)
    
  # Amostra de 20 filmes

  n_filmes = len(filmes_input)
  score = score[(n_filmes+1):20]
  indices = [i[0] for i in score]
  
  #return (score)
  return (mapear.iloc[indices])

In [ ]:
sistema2(['Toy Story','Cars',"A Bug's Life",'Toy Story 2'])

title
Cars 2                                        17551
Creature Comforts                              3336
Toy Story 3                                   15519
Fallen Art                                    12470
Idiots and Angels                             17254
Tin Toy                                       19301
Red's Dream                                   19355
The Bugs Bunny/Road Runner Movie              15734
Kung Fu Panda: Secrets of the Furious Five    13654
Mutant Aliens                                 10076
Bye-Bye Bin Laden                             14447
Darkness/Light/Darkness                       14515
The Employment                                16188
The Ugly Duckling and Me!                     11690
Hoodwinked Too! Hood VS. Evil                 19804
dtype: int64

In [ ]:
sistema('Toy Story')

title
Toy Story 2                                    3024
Toy Story 3                                   15519
Creature Comforts                              3336
The Bugs Bunny/Road Runner Movie              15734
Monster House                                 11209
Cars 2                                        17551
Fallen Art                                    12470
Kung Fu Panda: Secrets of the Furious Five    13654
Spiderman: The Ultimate Villain Showdown      15209
Idiots and Angels                             17254
Tin Toy                                       19301
Red's Dream                                   19355
Meet the Deedles                               1738
Mr. Bug Goes to Town                          18557
The Ugly Duckling and Me!                     11690
The Wrong Trousers                             1131
Jungle 2 Jungle                                1437
Christmas with the Kranks                      8334
A Boy Named Charlie Brown                     11369
dtype: